In [1]:
import pandas as pd

In [2]:
# On charge les données issues de nos trois tables
all_values = pd.read_excel("data/Valeurs_foncières.xlsx")
all_cities = pd.read_excel("data/donnees_communes.xlsx")
all_cities2 = pd.read_excel("data/fr-esr-referentiel-geographique.xlsx")

/home/vincent/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
values = all_values[["Date mutation", "Valeur fonciere", "No voie", "Type de voie", 
                     "Voie", "Code ID commune", "Code postal", "Commune", "Code departement", 
                     "Type local", "Surface reelle bati", "Nombre pieces principales"]]

In [4]:
cities = all_cities[["CODREG", "REG", "PTOT", "COM"]]

In [5]:
cities2 = all_cities2[["regrgp_nom", "reg_nom", "dep_nom", "dep_code", "reg_code", "com_nom", "com_nom_maj"]]

In [6]:
# Création du dataframe pour la table Region
region_table = cities2[["regrgp_nom", "reg_nom", "reg_code"]]
print(region_table.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38916 entries, 0 to 38915
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   regrgp_nom  38916 non-null  object
 1   reg_nom     38916 non-null  object
 2   reg_code    38916 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 912.2+ KB
None


In [7]:
region_table = region_table.drop_duplicates(subset=["reg_code"])
region_table.rename(columns={"reg_nom": "REG", "reg_code" : "CODREG"}, inplace=True)

In [8]:
# Création du dataframe pour la table Departement
dep_table = all_cities2[["dep_nom", "dep_code", "reg_code"]]
dep_table = dep_table.drop_duplicates(subset="dep_code")
dep_table.rename(columns={"dep_code" : "Code département", "reg_code": "CODREG"}, inplace=True)

In [32]:
# Création de la dataframe pour la table Commune, en plusieurs merges intermédiaires
intermediate = cities2.merge(how="left", right=cities, left_on=["com_nom", "reg_code"], right_on=["COM", "CODREG"])

intermediate2 = intermediate.merge(how="left", right=values, right_on=["Commune", "Code departement"], left_on=["com_nom_maj", "dep_code"])

commune_table = intermediate2.drop_duplicates(subset=["com_nom_maj", "dep_code"])
commune_table = commune_table[["Code ID commune", "Code postal", "com_nom_maj", "dep_code", "PTOT"]]
commune_table.rename(columns={"Code ID commune": "code commune", "Code postal": "code_postal", 
                              "com_nom_maj": "nom_commune", "dep_code": "code_departement"}, inplace=True)

In [ ]:
# Création de la dataframe pour la table Bien
bien = values[[
